In [1]:
import requests
from datetime import datetime
from dateutil import tz
import pandas as pd
import re


class CalendarAPI:
    def __init__(self, base_url, params=None, cookies=None, headers=None):
        self.base_url = base_url
        self.cookies = cookies or {}
        self.headers = headers or {}
        self.params = params or {}

    def fetch_events(self, year, month):
        json_data = [{
            'index': 0,
            'methodname': 'core_calendar_get_calendar_monthly_view',
            'args': {
                'year': str(year),
                'month': str(month),
                'courseid': 1,
                'day': 1,
                'view': 'month',
            },
        }]

        response = requests.post(self.base_url, params=self.params,
                                 cookies=self.cookies, headers=self.headers, json=json_data)
        return response.json()[0].get('data', {})


class EventParser:
    @staticmethod
    def get_date(input_string):
        pattern = r'(\d{2}:\d{2})'
        match = re.search(pattern, input_string)
        return match.group(1) if match else "No match found"

    @staticmethod
    def date_formatter(date_str, lang='ru'):
        if lang == 'ru':
            res = [val.capitalize()
                   for val in date_str.replace('События - ', '').split()]
            return f"{res[0]}, {' '.join(res[1:])}"
        elif lang == 'en':
            return ' '.join([val.strip().capitalize() for val in date_str.replace('events', '').strip().split()])

    @staticmethod
    def flatten(lst):
        return [item for sublist in lst for item in sublist]


class EventDataProcessor:
    def __init__(self, data):
        self.data = data

    def extract_events(self):
        events = [[day for day in week.get('days', []) if day.get(
            'hasevents')] for week in self.data.get('weeks', [])]
        events_flat = EventParser.flatten(events)

        result = []
        for event in events_flat:
            for e in event.get('events', [{}]):
                result.append([
                    event.get('viewdaylink', ''),
                    str(datetime.fromtimestamp(event.get('timestamp'), tz=tz.gettz("GMT+3"))),
                    event.get('viewdaylinktitle', ''),
                    e.get('activityname', ''),
                    e.get('viewurl'),
                    e.get('url', ''),
                    e.get('description', ''),
                    e.get('formattedtime'),
                    e.get('course', {}).get('fullnamedisplay'),
                    e.get('course', {}).get('viewurl', '')
                ])
        return result


class CalendarOutput:
    def __init__(self, events):
        self.events = events
        self.df = pd.DataFrame(events)
        self.df.columns = ['calendar_link', 'date', 'date_str', 'name', 'calendar',
                           'hw_link', 'description', 'due_time', 'subj', 'subj_link']

    def generate_output(self):
        output = f'Время последнего обновления: {str(datetime.now())[:19]} \n'
        self.df = self.df.sort_values('date').reset_index(drop=True)

        for dt in self.df.date_str.unique():
            output += '\n' + EventParser.date_formatter(dt) + '\n'
            cur_df = self.df[self.df.date_str == dt]
            for idx in range(len(cur_df)):
                output += f'{cur_df.iloc[idx].subj} {cur_df.iloc[idx].subj_link}  \n'
                output += f"{cur_df.iloc[idx]['name']} {cur_df.iloc[idx].hw_link} \n"
                output += f'До {EventParser.get_date(cur_df.iloc[idx].due_time)} \n'

        return output


if __name__ == "__main__":
    cookies = {
        'YOUR_COOKIES':''
    }
    headers = {
        'YOUR_HEADERS': ''
    }
    params = {
        'sesskey': 'YOUR_SESSKEY',
        'info': 'core_calendar_get_calendar_monthly_view',
    }
    api = CalendarAPI(base_url='https://edu.hse.ru/lib/ajax/service.php',
                      params=params, cookies=cookies, headers=headers)
    events = []
    for month in (9, 10):
        data = api.fetch_events(year=datetime.now().year, month=month)
        processor = EventDataProcessor(data)
        events.extend(processor.extract_events())
    calendar_output = CalendarOutput(events)
    output = calendar_output.generate_output()

    print(output)

Время последнего обновления: 2024-09-11 12:05:57 

Суббота, 14 Сентября
Python для инженерии данных (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211078  
Домашнее задание 1 https://edu.hse.ru/mod/assign/view.php?id=1280150 
До 23:59 

Понедельник, 16 Сентября
Информационные системы (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211074  
Домашнее задание по теме 1 «Введение в информационные системы» https://edu.hse.ru/mod/assign/view.php?id=1296563 
До 16:00 
Семинар наставника (2024/2025 модули: 1,2,3,4) https://edu.hse.ru/course/view.php?id=211075  
Паспорт компетенций https://edu.hse.ru/mod/assign/view.php?id=1290223 
До 23:59 

Понедельник, 23 Сентября
Информационные системы (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211074  
Домашнее задание по теме 2 «Методологии и инструменты создания информационных систем» https://edu.hse.ru/mod/assign/view.php?id=1296574 
До 16:00 

Суббота, 28 Сентября
Python для инженерии данных (2024/2025 мо

In [ ]:
# tg @nikitaworonov для доработок
# TODO Подумать можно ли это все дело как - то удобно кидать в ТГ
# TODO Может развернуть и захостить все это в кроне
# TODO Может по файликам лучше в проект отдельные это дело раскидать